# Model training notebook
This notebook is made for training the TDNN models from XVector.py module

In this notebook we carry out the grid search over the model's hyperparameters defined in grid dictionary and save the results as tensorboard runs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 2060
Using device: cuda


In [3]:
import pandas as pd
import numpy as np
import mne
import os
import matplotlib.pyplot as plt

In [4]:
from eeg_lib.commons.constant import DATASETS_FOLDER
from eeg_lib.data.data_loader.EEGDataExtractor import EEGDataExtractor

In [5]:
from eeg_lib.utils.engine import create_user_profiles

In [6]:
from eeg_lib.utils.helpers import compute_genuine_imposter_distances, compute_threshold_metrics, compute_f1_vs_threshold, split_test_data_for_verification

In [7]:
from eeg_lib.utils.visualisations import plot_distance_distribution_on_ax, plot_threshold_metrics, plot_f1_vs_threshold, plot_distance_distribution_return, plot_f1_vs_threshold_return, plot_threshold_metrics_return

In [8]:
DATA_DIR = f"{DATASETS_FOLDER}/Kolory/"

extractor = EEGDataExtractor(data_dir=DATA_DIR, tmax=5.0, lfreq=7.0, tmin=-2.0)
eeg_df, participants_info = extractor.extract_dataframe()

Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/011595b1@1651.fif...
    Range : 0 ... 207624 =      0.000 ...   830.496 secs
Ready.
Reading 0 ... 207624  =      0.000 ...   830.496 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/011595b1@1651.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/022e8467@1910.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 'red', 'yellow']
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1751 original time points ...
1 bad epochs dropped
Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/06f240e9@1215.fif...
    Range : 

...

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 'red', 'yellow']
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1751 original time points ...
0 bad epochs dropped
Opening raw data file C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/fd8a3308@1135.fif...
    Range : 0 ... 228724 =      0.000 ...   914.896 sec

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/fd8a3308@1135.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 413 samples (1.652 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband

C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib\data\data_loader\EEGDataExtractor.py:53: RuntimeWarning: This filename (C:\Users\JS\Desktop\Neuron\NeuroGuard\artificial-intelligence\eeg_lib/data/datasets/Kolory/ffae50df@1712.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg = mne.io.read_raw_fif(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['blue', 'break', 'gray', 'green', 'red', 'yellow']
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1751 original time points ...
0 bad epochs dropped


In [9]:
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from matplotlib.colors import ListedColormap

In [10]:
from sklearn.model_selection import ParameterGrid, GroupKFold

In [11]:
import torch.nn as nn

In [12]:
from eeg_lib.data.data_loader.TDNNFeatures import extract_features, extract_psd_features
from eeg_lib.data.TDNNDataset import TDNNDataset, get_dataset
from eeg_lib.models.verification.XVector import XVectorEmbeddingModel
from eeg_lib.losses.ProxyNCALoss import ProxyNCALoss
from eeg_lib.utils.visualisations import plot_tsne
from eeg_lib.utils.visualisations import create_handles
from eeg_lib.utils.helpers import split_train_test
from eeg_lib.models.similarity.centroids import SimilarityCentroidsVerifier, get_accuracy

In [13]:
from torch.utils.tensorboard import SummaryWriter

In [14]:
from torch.optim.lr_scheduler import StepLR, ExponentialLR, ReduceLROnPlateau

In [15]:
from eeg_lib.models.verification.XVector import get_ecapa_model, get_standard_model, pretrain, fine_tune, create_embeddings, fine_tune_arcface

grid with defined available hyperparameters for the TDNN model (ECAPA_TDNNv2 in particular)

In [16]:
K = 5
RUNS_FOLDER = "runs_ECAPA"
APPENDIX = "ECAPAv7"

In [17]:
grid = {
    "batch_size" : [64],
    "softmax_learning_rate" : [0.001],
    "proxy_learning_rate" : [0.001],
    "softmax_epochs" : [20],
    "proxy_epochs" : [80],
    "softmax_learning_rate_decay" : [0.95],
    "proxy_learning_rate_decay" : [0.95],
    "augmentation" : [True],
    "std" : [0.02],
    "embedding_dim" : [256],
    "dropout_rate" : [0.25],
    "scale" : [10],
    "margin" : [0.1],
    "layer1_filters" : [512],
    "layer2_filters" : [512],
    "layer3_filters" : [1024],
    "layer4_filters" : [1024],
    "layer5_filters" : [1500],
    "layer_1_dilatation" : [1],
    "layer_2_dilatation" : [2],
    "layer_3_dilatation" : [3],
    "layer_1_stride" : [1],
    "layer_2_stride" : [1],
    "layer_3_stride" : [2],
    "no_norm" : [True]
}

Data loading and preprocessing

In [18]:
X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = split_train_test(eeg_df=eeg_df,test_size=0.1, random_state=1234)

Training set participants: ['022e8467@1910' 'fd8a3308@1135' '39285860@1825' '25d0bdb3@1318'
 'd87e1bd3@1806' '54e60118@1339' '548fd734@1628' 'bf2d2193@1638'
 'e08138e2@1731' '011595b1@1651' '06f240e9@1215' 'f82b5699@1757'
 'e43a9f9f@0941' '8dca0725@1418' '2882ae26@1441' '6e542bc2@0845'
 'e283301e@1606' '51ec2c20@0923' '446b3735@1618' '8bd3032e@1746'
 'ffae50df@1712' '9e8bae0e@1828' '541c91f2@1456' 'b34b1427@0906'
 '3033b74a@1626' '90441f44@1643' '6d9a8b86@1613']
Test set participants: ['36eea4bb@1519' '46607ce4@1717' '2718372d@1400']
Training labels: ['011595b1@1651' '022e8467@1910' '06f240e9@1215' '25d0bdb3@1318'
 '2882ae26@1441' '3033b74a@1626' '39285860@1825' '446b3735@1618'
 '51ec2c20@0923' '541c91f2@1456' '548fd734@1628' '54e60118@1339'
 '6d9a8b86@1613' '6e542bc2@0845' '8bd3032e@1746' '8dca0725@1418'
 '90441f44@1643' '9e8bae0e@1828' 'b34b1427@0906' 'bf2d2193@1638'
 'd87e1bd3@1806' 'e08138e2@1731' 'e283301e@1606' 'e43a9f9f@0941'
 'f82b5699@1757' 'fd8a3308@1135' 'ffae50df@1712']
Tes

In [19]:
epoch_length = X_train_tmp[0].shape[1]
div, mod = divmod(epoch_length, 50)
truncated_length = div*50

In [20]:
extracted_X_train = []
for epoch in X_train_tmp:
    extracted_X_train.append(extract_features(epoch, fs=250, trunc=truncated_length).T) 

In [21]:
extracted_X_test = []
for epoch in X_test_tmp:
    extracted_X_test.append(extract_features(epoch, fs=250, trunc=truncated_length).T)

In [22]:
le_train = LabelEncoder()
le_train.fit(y_train_tmp)
y_train_encoded = le_train.transform(y_train_tmp)

le_test = LabelEncoder()
le_test.fit(y_test_tmp)
y_test_encoded = le_test.transform(y_test_tmp)

In [23]:
X_train = np.array(extracted_X_train)
X_test = np.array(extracted_X_test)

In [24]:
scalers = {}
X_train_norm = np.empty_like(X_train)
X_test_norm = np.empty_like(X_test)

In [25]:
# scaling per feature
for f in range(X_train.shape[1]):
    scalers[f] = StandardScaler().fit(X_train[:, f, :])
    X_train_norm[:, f,:] = scalers[f].transform(X_train[:, f, :])
    X_test_norm[:, f, :] = scalers[f].transform(X_test[:, f, :])

In [26]:
num_train_classes = len(np.unique(y_train_encoded))

In [27]:
groups = np.array(y_train_encoded)
X = X_train_norm
y = np.array(y_train_encoded)

In [28]:
def make_loader_from_indecies(X_arr,y_arr, indicies, hparams):
    X_sub = X_arr[indicies]
    y_sub = y_arr[indicies]
    loader = get_dataset(hparams, X_sub, y_sub)
    return loader, X_sub, y_sub

In [29]:
splitter = GroupKFold(n_splits=K)
splits = splitter.split(X,y,groups=groups)

In [30]:
custom_colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
                 '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff',
                 '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1',
                 '#000075', '#808080', '#ffffff', '#000000', '#a9a9a9', '#ff69b4',
                 '#b0e0e6', '#32cd32', '#ff4500', '#da70d6', '#ff1493', '#7fffd4']
cmap = ListedColormap(custom_colors[:30])

custom_colors = ['#e6194b',  # Red
                 '#3cb44b',  # Green
                 '#4363d8',  # Blue
                 '#ffe119',  # Yellow
                 '#911eb4',  # Purple
                 '#f58231']  # Orange

cmap2 = ListedColormap(custom_colors[:6])

In [31]:
train_handles = create_handles(y_train_tmp, cmap)
test_handles = create_handles(y_test_tmp, cmap2)

In the tensorboard run we save:
- pretaraining loss
- pretraining accuracy
- finetuning loss
- centroid based accuracy metric for train and test data
- test and train TSNE figures
- genuine vs imposter plots for both cosine and euclidean distances train and test
- threshold values, f1 score at threshold, accuracy at threshold, for cosine, euclidean metrics, train and test data
- plots for f1 vs threshold
- plots for accuracy vs threshold

In [32]:
import matplotlib.patches as mpatches
from matplotlib import cm

In [34]:
for run_idx, hparams in enumerate(ParameterGrid(grid), start=1):
    print("RUN NUMBER: {}".format(run_idx))
    run_name = f"run_{run_idx}"
    writer = SummaryWriter(log_dir=f"{RUNS_FOLDER}/{run_name}_{APPENDIX}")
    # print(hparams)
    writer.add_hparams(hparams, {})
    
    avg_fold_acc = 0
    avg_fold_f1 = 0
    fold_idx = 0
    for train_idx, val_idx in splits:
        fold_idx += 1
        print(f"FOLD: {fold_idx}")
    
        _, X_tr_sub, y_tr_sub = make_loader_from_indecies(X, y, train_idx, hparams) 
        _, X_val_sub, y_val_sub = make_loader_from_indecies(X,y,val_idx,hparams)
        
        # print("unique labels in train:", np.unique(y_tr_sub)[:10], "n_unique:", len(np.unique(y_tr_sub)))
        # print("label max:", y_tr_sub.max(), "expected max:", len(np.unique(y_tr_sub))-1)
        
        unique_train = np.unique(y_tr_sub)
        train_label_map = {old:new for new, old in enumerate(unique_train)}
        y_tr_mapped = np.array([train_label_map[x] for x in y_tr_sub], dtype=int)
        num_classes_fold =len(unique_train)
        
        train_loader = get_dataset(hparams, X_tr_sub, y_tr_mapped)
        val_loader = get_dataset(hparams, X_val_sub, y_val_sub)
        
        # Sanity-check: labels in the train loader must be within [0, num_classes_fold-1]
        # for bx, by in train_loader:
        #     print("train batch labels min,max:", by.min().item(), by.max().item())
        #     assert by.min().item() >= 0 and by.max().item() < num_classes_fold, "Mapped labels out of range!"
        #     break
        
        model = pretrain(hparams, device, X_tr_sub.shape[1], len(np.unique(y_tr_sub)), train_loader, writer, "ECAPA2", fold_idx).to(device)
        model = fine_tune_arcface(model, hparams, device, train_loader, len(np.unique(y_tr_sub)), writer, fold=fold_idx).to(device)
    
    # checkpoint = {
    #     "model_state_dict": model.state_dict()
    # }
    # torch.save(checkpoint, "saved_models/ECAPA2/init_model_for_featuregrid.pth")
        model.to('cpu')
        embd_train, embd_val = create_embeddings(model, X_tr_sub, X_val_sub,hparams)
    
        centroid_train_acc, centroid_val_acc = get_accuracy(embd_train, embd_val, y_tr_sub, y_val_sub)
    
        writer.add_scalar(f"Final_Centroid_Accuracy_fold_{fold_idx}/train", centroid_train_acc)
        writer.add_scalar(f"Final_Centroid_Accuracy_fold_{fold_idx}/val", centroid_val_acc)
    
        tsne_train = TSNE(n_components=2, random_state=42).fit_transform(embd_train)
        tsne_val = TSNE(n_components=2, random_state=42).fit_transform(embd_val)
    
        scaler = MinMaxScaler(feature_range=(0,1))
        tsne_train_scaled = scaler.fit_transform(tsne_train)
        tsne_val_scaled = scaler.transform(tsne_val)
        
        uniq_train, inv_train = np.unique(y_tr_sub, return_inverse=True)
        uniq_val,   inv_val   = np.unique(y_val_sub, return_inverse=True)
        
        C_train = len(uniq_train)
        C_val   = len(uniq_val)
        
        cmap_train = cm.get_cmap('tab20', C_train) if C_train <= 20 else cm.get_cmap('hsv', C_train)
        cmap_val   = cm.get_cmap('tab20', C_val)   if C_val   <= 20 else cm.get_cmap('hsv', C_val)
        
        train_patches = [mpatches.Patch(color=cmap_train(i), label=str(uniq_train[i])) for i in range(C_train)]
        val_patches   = [mpatches.Patch(color=cmap_val(i),   label=str(uniq_val[i]))   for i in range(C_val)]
        
    
        fig_train = plot_tsne(tsne_train_scaled,
              cmap_train,
              inv_train,
              handles=train_patches,
              alpha=0.5,
              title="TSNE Visualization training data XVector",
              xlabel="TSNE embedding dimension 1",
              ylabel="TSNE embedding dimension 2",
              centroids=None,
              return_fig=True)
    # plt.show(fig_train)
        writer.add_figure(f"tsne_train_fold_{fold_idx}", fig_train)
        fig_val = plot_tsne(tsne_val_scaled,
              cmap_val,
              inv_val,
              handles=val_patches,
              alpha=0.5,
              title="TSNE Visualization test data XVector",
              xlabel="TSNE embedding dimension 1",
              ylabel="TSNE embedding dimension 2",
              centroids=None, return_fig=True )
        # plt.show(fig_test)
        embd_unit = embd_train / np.linalg.norm(embd_train, axis=1, keepdims=True)
        writer.add_figure(f"tsne_val_fold_{fold_idx}", fig_val)
    
        fig = plot_distance_distribution_return(
            embeddings=embd_unit,
            participant_ids=y_tr_sub,
            distance_type="euclidean",
            bins=30
        )
    # plt.show(fig)
        writer.add_figure(f"distance_distribution_euclidean_fold_{fold_idx}", fig)
    
        fig = plot_distance_distribution_return(
            embeddings=embd_train,
            participant_ids=y_tr_sub,
            distance_type="cosine",
            bins=30
        )
    # plt.show(fig)
        writer.add_figure("distance_distribution_cosine", fig)
    
        # train
        user_profiles = create_user_profiles(embd_train, y_tr_sub)
        for metric in ("euclidean", "cosine"):
            genuine_dists, imposter_dists = compute_genuine_imposter_distances(
                    embeddings=embd_train ,
                    ids=y_tr_sub,
                    user_profiles=user_profiles,
                    distance_metric=metric
                )
            
            (
                thresholds, fnr_list, fpr_list, acc_list,
                best_T, best_fnr, best_fpr, best_acc
            ) = compute_threshold_metrics(genuine_dists, imposter_dists, num_thresholds=200)
            
            writer.add_scalar(f"{metric}_best_threshold_train_fold_{fold_idx}", best_T)
            writer.add_scalar(f"{metric}_best_acc_train_fold_{fold_idx}", best_acc)
            writer.add_scalar(f"{metric}_FNR/FPR_threshold_train_fold_{fold_idx}", best_fnr)
            

            
            
            fig = plot_threshold_metrics_return(
                thresholds, fnr_list, fpr_list, acc_list,
                best_T, best_fnr, best_fpr, best_acc
            )
            # plt.show(fig)
            writer.add_figure(f"threshold_metrics_train_{metric}_fold_{fold_idx}", fig)
            
            thresholds, f1_list, best_T, best_f1 = compute_f1_vs_threshold(
                genuine_dists, imposter_dists, num_thresholds=300
            )
            writer.add_scalar(f"{metric}_best_f1_train_fold_{fold_idx}", best_f1)
            

            
            fig = plot_f1_vs_threshold_return(thresholds, f1_list, best_T, best_f1)
            writer.add_figure(f"f1_vs_threshold_train_{metric}_fold_{fold_idx}", fig)
            # plt.show(fig)
    

        # vaL
        profile_embd, profile_ids, verify_embd, verify_ids = split_test_data_for_verification(
            embd_val, y_val_sub, profile_ratio=0.6
        )
        
        val_user_profiles = create_user_profiles(profile_embd, profile_ids)
        for metric in ("euclidean", "cosine"):
        
            genuine_dists, imposter_dists = compute_genuine_imposter_distances(
                embeddings=verify_embd,
                ids=verify_ids,
                user_profiles=val_user_profiles,
                distance_metric=metric
            )
            
            (
                test_thresholds, test_fnr_list, test_fpr_list, test_acc_list,
                test_best_T, test_best_fnr, test_best_fpr, test_best_acc
            ) = compute_threshold_metrics(genuine_dists, imposter_dists, num_thresholds=200)
            writer.add_scalar(f"{metric}_best_threshold_val_fold_{fold_idx}", test_best_T)
            writer.add_scalar(f"{metric}_best_acc_val_fold_{fold_idx}", test_best_acc)
            writer.add_scalar(f"{metric}_FNR/FPR_th0reshold_val_fold_{fold_idx}", test_best_fnr)
            
            
            if metric == "cosine":
                avg_fold_acc += test_best_acc
            
            fig = plot_threshold_metrics_return(
                test_thresholds, test_fnr_list, test_fpr_list, test_acc_list,
                test_best_T, test_best_fnr, test_best_fpr, test_best_acc
            )
            # plt.show(fig)
            writer.add_figure(f"threshold_metrics_{metric}_val_fold_{fold_idx}", fig)
            
            test_thresholds_f1, test_f1_list, test_best_T_f1, test_best_f1 = compute_f1_vs_threshold(
                genuine_dists, imposter_dists, num_thresholds=300
            )
            writer.add_scalar(f"{metric}_best_f1_val_fold_{fold_idx}", test_best_f1)
            
            if metric == "cosine":
                avg_fold_f1 += test_best_f1
            
            fig = plot_f1_vs_threshold_return(test_thresholds_f1, test_f1_list, test_best_T_f1, test_best_f1)
            # plt.show(fig)
            writer.add_figure(f"f1_vs_threshold_val_{metric}_fold_{fold_idx}", fig)
    avg_fold_acc = avg_fold_acc / fold_idx
    avg_fold_f1 = avg_fold_f1 / fold_idx
    writer.add_scalar("Average Fold Accuracy cosine", avg_fold_acc)
    writer.add_scalar("Average Fold F1 cosine", avg_fold_f1)
    writer.close()                                                                                    

RUN NUMBER: 1
FOLD: 1
[Pretrain] Epoch 1/20  Loss=2.9046  Acc=0.0853
[Pretrain] Epoch 2/20  Loss=2.7604  Acc=0.0908
[Pretrain] Epoch 3/20  Loss=2.6993  Acc=0.0920
[Pretrain] Epoch 4/20  Loss=2.6454  Acc=0.1175
[Pretrain] Epoch 5/20  Loss=2.4893  Acc=0.1720
[Pretrain] Epoch 6/20  Loss=2.3206  Acc=0.2168
[Pretrain] Epoch 7/20  Loss=2.2091  Acc=0.2287
[Pretrain] Epoch 8/20  Loss=2.1193  Acc=0.2696
[Pretrain] Epoch 9/20  Loss=2.0364  Acc=0.3084
[Pretrain] Epoch 10/20  Loss=1.9465  Acc=0.3719
[Pretrain] Epoch 11/20  Loss=1.8471  Acc=0.4231
[Pretrain] Epoch 12/20  Loss=1.7616  Acc=0.4434
[Pretrain] Epoch 13/20  Loss=1.6483  Acc=0.4812
[Pretrain] Epoch 14/20  Loss=1.5395  Acc=0.5143
[Pretrain] Epoch 15/20  Loss=1.4199  Acc=0.5778
[Pretrain] Epoch 16/20  Loss=1.3099  Acc=0.6347
[Pretrain] Epoch 17/20  Loss=1.1887  Acc=0.6856
[Pretrain] Epoch 18/20  Loss=1.0875  Acc=0.7208
[Pretrain] Epoch 19/20  Loss=1.0066  Acc=0.7428
[Pretrain] Epoch 20/20  Loss=0.9381  Acc=0.7619
[ArcFace Fine‑tune] Epoch 1

C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:63: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_train = cm.get_cmap('tab20', C_train) if C_train <= 20 else cm.get_cmap('hsv', C_train)
C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:64: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_val   = cm.get_cmap('tab20', C_val)   if C_val   <= 20 else cm.get_cmap('hsv', C_val)


FOLD: 2
[Pretrain] Epoch 1/20  Loss=2.8894  Acc=0.1134
[Pretrain] Epoch 2/20  Loss=2.5427  Acc=0.2028
[Pretrain] Epoch 3/20  Loss=2.2224  Acc=0.3729
[Pretrain] Epoch 4/20  Loss=1.8758  Acc=0.5279
[Pretrain] Epoch 5/20  Loss=1.5602  Acc=0.6057
[Pretrain] Epoch 6/20  Loss=1.2756  Acc=0.6879
[Pretrain] Epoch 7/20  Loss=1.0346  Acc=0.7705
[Pretrain] Epoch 8/20  Loss=0.8514  Acc=0.8240
[Pretrain] Epoch 9/20  Loss=0.6941  Acc=0.8650
[Pretrain] Epoch 10/20  Loss=0.5532  Acc=0.9013
[Pretrain] Epoch 11/20  Loss=0.4503  Acc=0.9239
[Pretrain] Epoch 12/20  Loss=0.3641  Acc=0.9402
[Pretrain] Epoch 13/20  Loss=0.3072  Acc=0.9543
[Pretrain] Epoch 14/20  Loss=0.2515  Acc=0.9617
[Pretrain] Epoch 15/20  Loss=0.2162  Acc=0.9692
[Pretrain] Epoch 16/20  Loss=0.1836  Acc=0.9743
[Pretrain] Epoch 17/20  Loss=0.1484  Acc=0.9829
[Pretrain] Epoch 18/20  Loss=0.1395  Acc=0.9825
[Pretrain] Epoch 19/20  Loss=0.1154  Acc=0.9894
[Pretrain] Epoch 20/20  Loss=0.1173  Acc=0.9847
[ArcFace Fine‑tune] Epoch 1/80  Loss=0.68

C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:63: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_train = cm.get_cmap('tab20', C_train) if C_train <= 20 else cm.get_cmap('hsv', C_train)
C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:64: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_val   = cm.get_cmap('tab20', C_val)   if C_val   <= 20 else cm.get_cmap('hsv', C_val)


FOLD: 3
[Pretrain] Epoch 1/20  Loss=2.8776  Acc=0.0940
[Pretrain] Epoch 2/20  Loss=2.5699  Acc=0.1561
[Pretrain] Epoch 3/20  Loss=2.2798  Acc=0.2472
[Pretrain] Epoch 4/20  Loss=2.0333  Acc=0.3658
[Pretrain] Epoch 5/20  Loss=1.7454  Acc=0.5465
[Pretrain] Epoch 6/20  Loss=1.4339  Acc=0.6848
[Pretrain] Epoch 7/20  Loss=1.1338  Acc=0.7956
[Pretrain] Epoch 8/20  Loss=0.8896  Acc=0.8550
[Pretrain] Epoch 9/20  Loss=0.6960  Acc=0.8888
[Pretrain] Epoch 10/20  Loss=0.5687  Acc=0.9037
[Pretrain] Epoch 11/20  Loss=0.4490  Acc=0.9343
[Pretrain] Epoch 12/20  Loss=0.3627  Acc=0.9464
[Pretrain] Epoch 13/20  Loss=0.3078  Acc=0.9588
[Pretrain] Epoch 14/20  Loss=0.2421  Acc=0.9692
[Pretrain] Epoch 15/20  Loss=0.2130  Acc=0.9723
[Pretrain] Epoch 16/20  Loss=0.1865  Acc=0.9757
[Pretrain] Epoch 17/20  Loss=0.1519  Acc=0.9827
[Pretrain] Epoch 18/20  Loss=0.1362  Acc=0.9849
[Pretrain] Epoch 19/20  Loss=0.1164  Acc=0.9898
[Pretrain] Epoch 20/20  Loss=0.1034  Acc=0.9912
[ArcFace Fine‑tune] Epoch 1/80  Loss=0.66

C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:63: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_train = cm.get_cmap('tab20', C_train) if C_train <= 20 else cm.get_cmap('hsv', C_train)
C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:64: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_val   = cm.get_cmap('tab20', C_val)   if C_val   <= 20 else cm.get_cmap('hsv', C_val)


FOLD: 4
[Pretrain] Epoch 1/20  Loss=3.0610  Acc=0.0594
[Pretrain] Epoch 2/20  Loss=2.9485  Acc=0.0798
[Pretrain] Epoch 3/20  Loss=2.5922  Acc=0.1524
[Pretrain] Epoch 4/20  Loss=2.3652  Acc=0.1881
[Pretrain] Epoch 5/20  Loss=2.2258  Acc=0.2515
[Pretrain] Epoch 6/20  Loss=2.1336  Acc=0.2683
[Pretrain] Epoch 7/20  Loss=2.0011  Acc=0.3235
[Pretrain] Epoch 8/20  Loss=1.9169  Acc=0.3329
[Pretrain] Epoch 9/20  Loss=1.8428  Acc=0.3533
[Pretrain] Epoch 10/20  Loss=1.7416  Acc=0.3843
[Pretrain] Epoch 11/20  Loss=1.6756  Acc=0.4178
[Pretrain] Epoch 12/20  Loss=1.5451  Acc=0.4876
[Pretrain] Epoch 13/20  Loss=1.4306  Acc=0.5396
[Pretrain] Epoch 14/20  Loss=1.2985  Acc=0.6269
[Pretrain] Epoch 15/20  Loss=1.1703  Acc=0.6826
[Pretrain] Epoch 16/20  Loss=1.0391  Acc=0.7458
[Pretrain] Epoch 17/20  Loss=0.9319  Acc=0.7838
[Pretrain] Epoch 18/20  Loss=0.8179  Acc=0.8303
[Pretrain] Epoch 19/20  Loss=0.7321  Acc=0.8515
[Pretrain] Epoch 20/20  Loss=0.6552  Acc=0.8686
[ArcFace Fine‑tune] Epoch 1/80  Loss=1.26

C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:63: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_train = cm.get_cmap('tab20', C_train) if C_train <= 20 else cm.get_cmap('hsv', C_train)
C:\Users\JS\AppData\Local\Temp\ipykernel_26052\532567452.py:64: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap_val   = cm.get_cmap('tab20', C_val)   if C_val   <= 20 else cm.get_cmap('hsv', C_val)
